# 1. 허깅페이스를 통한 한국어 말뭉치 데이터가 학습된, Llama 3.1 8b를 가져오기. (from :Bllossom)

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
import trl
from datasets import load_dataset

In [41]:
device = 'cuda' if torch.cuda.is_available() else "cpu"  #gpu 확인을 위해 사용
print(device)
print(torch.__version__)  # PyTorch 버전 확인
print(torch.version.cuda)  # PyTorch가 사용하는 CUDA 버전 확인
print(torch.cuda.is_available())  

cuda
2.6.0+cu124
12.4
True


In [61]:
model_name = 'Bllossom/llama-3.2-Korean-Bllossom-3B'
tokenizer = AutoTokenizer.from_pretrained(model_name)
### https://huggingface.co/Bllossom/llama-3.2-Korean-Bllossom-3B 참조.
model=  AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="cuda",
)

model.device

Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.94s/it]


device(type='cuda', index=0)

In [ ]:
# input_ids = tokenizer.apply_chat_template(
#     messages,
#     add_generation_prompt=True,
#     return_tensors="pt"
# ).to(model.device)


# 허깅페이스 카드에 나와있는 것은 챗 탬플릿이다. 다만 프롬프트를 세부적으로 작성하기 위해 프롬프트를 아래와 같이 직접 작성하고자 함.

In [55]:
model_id = 'Bllossom/llama-3.2-Korean-Bllossom-3B'

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
instruction = "철수가 20개의 연필을 가지고 있었는데 영희가 절반을 가져가고 민수가 남은 5개를 가져갔으면 철수에게 남은 연필의 갯수는 몇개인가요?"

messages = [
    {"role": "user", "content": f"{instruction}"}
    ]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.convert_tokens_to_ids("<|end_of_text|>"),
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=1024,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9
)

print(tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True))


c:\Users\user\miniconda3\envs\news_tts_project\Lib\site-packages\accelerate\utils\modeling.py:1536: UserWarning: Current model requires 128 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  4.45it/s]
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


철수가 20개의 연필을 가지고 있었고 영희가 절반을 가져갔을 때, 철수는 20 / 2 = 10개의 연필을 가지고 있습니다.

그리고 민수가 남은 5개를 가져갔을 때, 철수에게 남은 연필의 갯수는 10 - 5 = 5개입니다.

따라서 철수에게 남은 연필의 갯수는 5개입니다.


In [56]:
print(input_ids)

tensor([[128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,
             25,   6790,    220,   2366,     18,    198,  15724,   2696,     25,
            220,   1544,  13806,    220,   2366,     20,    271,   2675,    527,
            264,  11190,  15592,  18328,     13,   5321,   4320,    279,   1217,
            596,   4860,  47626,     13, 113783,  34804, 101003,  67119,  24486,
          15592, 101139,  30426,  25941,  95252,  29726, 119519,     13,  41820,
         110257, 109760,  19954, 112107, 108280, 104834, 102893, 111964,  34983,
          92769,     13, 128009, 128006,    882, 128007,    271, 107837, 123503,
            220,    508, 123590,  78453, 110174,  18359, 120693, 107417, 103170,
         101603, 105204,  20565, 110217, 101738,  18359,  89946,  20565,  35495,
         107138, 123503, 102484,  34804,    220,     20, 117594,  89946,  14705,
            242,  91040, 112521,  24140, 102244, 102484,  34804,  78453, 110174,
          21028,  17196,    

In [64]:
article ="""
블랙핑크 멤버들의 솔로 작품이 연달아 좋은 반응을 얻고 있다. 블랙핑크는 2023년 YG엔터테인먼트와 팀 활동 재계약을 맺으면서 개인 활동은 각자 새로운 둥지를 찾거나 꾸려서 하기로 했다. 이후 리사, 로제, 제니가 선보인 작품은 하나같이 세계 정상급 자본력과 기획력이 결합해야 나올 수 있는 퀄리티로 화제를 모았다. 올해 블랙핑크 컴백이 예상되는 가운데 지수가 2월 14일 미니 앨범 ‘아모르타주(AMORTAGE)’를 발매함으로써 네 멤버 모두 각자의 작업물을 내놓게 됐다.
‌타이틀곡 ‘어스퀘이크(earthquake)’는 듣다 보면 어쩐지 교묘하게 뒤엉킨 듯한 기분이 든다. 노래 각 부분이 서로 다른 방식으로 선명하게 인상을 남기기 때문이다. 호흡을 짚어가며 단음 위주로 흐르던 보컬이 “벗어나려 할수록…”으로 진입할 때면 갑자기 멜로디컬해지고, 심지어 1990년대 댄스가요 같은 인상마저 준다. “바우트 투 블로(Bout to blow)”라는 가사 그대로 터질 것만 같던 긴장이 시원하게 해소되는 건 후렴 부분부터다.
블랙핑크의 또 다른 세계
간단하되 오묘한 모티프로 음역을 옮겨가며 반복되는 후렴은 기하학적인 인상을 준다. 뜨거운 훅은 아니지만 선명하게 각인될 만하고, 무엇보다 안정감이 빼어나다. ‘이제 노래가 제자리로 돌아왔다’는 기분을 느끼게 한다. 거기에 시원하게 달려 나가는 클럽 스타일 하우스 비트가 결합한다. 기분은 ‘이제 춤출 일만 남았다’로 바뀐다. 마침 뮤직비디오 속 지수도 댄서들을 ‘거느리고’ 춤을 춘다. 이는 분명 팝 시장에서 하우스 음악이 제공할 수 있는 가장 좋은 점 가운데 하나다.
‌뮤직비디오는 K팝의 정수를 가져와 멋들어지게 표현해낸다. 전체주의적인 사회에서 권력과 감성이 대결한다는 설정, 질주하는 슈퍼카, 화려한 군무 같은 것은 K팝을 대외적으로 대표하는 이미지라 할 만하다. 어둡고 위압적인 사무실 풍경이나 거대한 스마트폰 자판 위에서 춤추는 댄서들 모습 등을 보면 금붙이 하나 없이도 ‘부티’가 흐른다는 느낌이 든다. 그리고 “그를 믿어도 될까”라는 질문 앞에서 가소롭다는 듯한 웃음을 띠며 가속페달을 밟아버리는 지수의 모습이 전하는 쾌감도 보통이 아니다.
‌이미 개인 작품을 선보인 다른 멤버들에 비해 지수의 노선은 좀 더 K팝적이다. K팝 정상에서 만들어낼 수 있는 것, 그러나 역시 블랙핑크 틀 안에서는 도무지 상상하기 어려운 어떤 것이다. 네 멤버의 네 방향 이정표가 이렇게 완성됐다. 이를 따라 확장될 새로운 블랙핑크의 세계를 기대하지 않기 어렵다.
"""

keyword ="블랙핑크"

prompt = f"""
요약 규칙:
1. {keyword}가 포함된 내용 위주로 요약해주세요.
2. {keyword}가 포함되지 않은 부분은 생략해주세요.
3. 5문장 이상으로 요약해주세요.
4. 같은 내용의 문장을 반복하지 마세요.
5. 모르는 부분은 대답하지 마세요.
6. 문장을 생성할 때 구어체로 이어지게 해주세요.

다음 문장을 요약해주세요.: \n
{article} \n

요약:
<|eot_id|>
"""

input_ids= tokenizer(prompt,
    return_tensors="pt")["input_ids"].to(device)

terminators = [
    tokenizer.convert_tokens_to_ids("<|end_of_text|>"),
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]


outputs = model.generate(
    input_ids,
    max_new_tokens=1024,
    do_sample=True,
    temperature=0.5,
    top_k=40,
    top_p=0.95
)

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [65]:
print(generated_text)


요약 규칙:
1. 블랙핑크가 포함된 내용 위주로 요약해주세요.
2. 블랙핑크가 포함되지 않은 부분은 생략해주세요.
3. 5문장 이상으로 요약해주세요.
4. 같은 내용의 문장을 반복하지 마세요.
5. 모르는 부분은 대답하지 마세요.
6. 문장을 생성할 때 구어체로 이어지게 해주세요.

다음 문장을 요약해주세요.: 


블랙핑크 멤버들의 솔로 작품이 연달아 좋은 반응을 얻고 있다. 블랙핑크는 2023년 YG엔터테인먼트와 팀 활동 재계약을 맺으면서 개인 활동은 각자 새로운 둥지를 찾거나 꾸려서 하기로 했다. 이후 리사, 로제, 제니가 선보인 작품은 하나같이 세계 정상급 자본력과 기획력이 결합해야 나올 수 있는 퀄리티로 화제를 모았다. 올해 블랙핑크 컴백이 예상되는 가운데 지수가 2월 14일 미니 앨범 ‘아모르타주(AMORTAGE)’를 발매함으로써 네 멤버 모두 각자의 작업물을 내놓게 됐다.
‌타이틀곡 ‘어스퀘이크(earthquake)’는 듣다 보면 어쩐지 교묘하게 뒤엉킨 듯한 기분이 든다. 노래 각 부분이 서로 다른 방식으로 선명하게 인상을 남기기 때문이다. 호흡을 짚어가며 단음 위주로 흐르던 보컬이 “벗어나려 할수록…”으로 진입할 때면 갑자기 멜로디컬해지고, 심지어 1990년대 댄스가요 같은 인상마저 준다. “바우트 투 블로(Bout to blow)”라는 가사 그대로 터질 것만 같던 긴장이 시원하게 해소되는 건 후렴 부분부터다.
블랙핑크의 또 다른 세계
간단하되 오묘한 모티프로 음역을 옮겨가며 반복되는 후렴은 기하학적인 인상을 준다. 뜨거운 훅은 아니지만 선명하게 각인될 만하고, 무엇보다 안정감이 빼어나다. ‘이제 노래가 제자리로 돌아왔다’는 기분을 느끼게 한다. 거기에 시원하게 달려 나가는 클럽 스타일 하우스 비트가 결합한다. 기분은 ‘이제 춤출 일만 남았다’로 바뀐다. 마침 뮤직비디오 속 지수도 댄서들을 ‘거느리고’ 춤을 춘다. 이는 분명 팝 시장에서 하우스 음악이 제공할 수 있는 가장 좋은 점 가운데 하나다.
‌뮤직비디오는 K팝의 정수를 가져와 멋들어지게 

In [53]:
print("Original eos_token_id:", tokenizer.eos_token_id)
print("Original bos_token_id:", tokenizer.bos_token_id)

eos_id = tokenizer.convert_tokens_to_ids("<|eot_id|>")
print("Using eos_id:", eos_id)


Original eos_token_id: 128009
Original bos_token_id: 128000
Using eos_id: 128009


# 2. summarize를 수행하기 위한 파인튜닝

In [ ]:
# 파인튜닝을 위한 데이터셋 
data = load_dataset("daekeun-ml/naver-news-summarization-ko") #네이버 뉴스 요약 데이터셋 : https://huggingface.co/datasets/daekeun-ml/naver-news-summarization-ko

c:\Users\user\miniconda3\envs\news_tts_project\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\datasets--daekeun-ml--naver-news-summarization-ko. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Generating test split: 100%|██████████| 2740/2740 [00:00<00:00, 23269.79 examples/

In [ ]:

#파인튜닝을 위한 데이터 프롬프트 문 작성 함수.
# def generate_prompts(example):
#     prompt_list = []
#     for i in range(len(example['document'])):
#         prompt_list.append(
# f"""<|begin_of_text|><|start_header_id|>user<|end_header_id|>다음 글을 요약해주세요:
# {example['document'][i]}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
# {example['summary'][i]}<|eot_id|>"""
#         )